In [6]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd

def fetch_box_score():
    hrefs = set()
    url = "https://www.basketball-reference.com/playoffs/NBA_2023.html"

    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Steps #1 to #9
    table_container = soup.find("div", {"id": "div_all_playoffs"})
    tbody = table_container.find("tbody")
    tr_toggleable = tbody.find_all("tr", {"class": "toggleable"})
    for trs in tr_toggleable:
        a_s = trs.find_all("a")
        [hrefs.add(x["href"]) for x in a_s]


    for href_link in list(hrefs):
        boxscore_link = f"https://www.basketball-reference.com{href_link}"
        boxscore_link = "https://www.basketball-reference.com/boxscores/202304160DEN.html"
        print(boxscore_link)
        response = requests.get(boxscore_link)
        soup = BeautifulSoup(response.content, 'html.parser')
        titles = set()
        tags_with_text = soup.find_all(lambda tag: " and Advanced Stats" in tag.text)
        lowest_level_tags = [tag for tag in tags_with_text if
                             not tag.find(lambda child: " and Advanced Stats" in child.text)]
        for x in lowest_level_tags:
            titles.add(x.text.replace("Table", "").strip())
        print(titles)

        rows = []
        for tbody in soup.find_all("tbody"):
            isStarter = True
            for trs in tbody.find_all("tr"):
                row = {}
                row["name"] = trs.find("th").text
                row["starter"] = isStarter
                if trs.find("th").text == "Reserves":
                    isStarter = False
                    continue
                for tds in trs.find_all("td"):
                    row[tds["data-stat"]] = tds.text
                rows.append(row)
                
        return rows
    
df = pd.DataFrame(fetch_box_score())


https://www.basketball-reference.com/boxscores/202304160DEN.html
{'Minnesota Timberwolves Basic and Advanced Stats', 'Denver Nuggets Basic and Advanced Stats'}


AttributeError: 'DataFrame' object has no attribute 'column_names'

In [9]:

basic_column_names = ['name', 'mp', 'fg', 'fga', 'fg_pct', 'fg3', 'fg3a', 'fg3_pct', 'ft','fta', 'ft_pct', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf','pts', 'plus_minus']
df_selected = df[basic_column_names].dropna()


                         name     mp fg fga fg_pct fg3 fg3a fg3_pct ft fta  \
0          Karl-Anthony Towns  29:34  5  15   .333   1    7    .143  0   2   
1             Anthony Edwards  27:41  6  15   .400   1    4    .250  5   6   
2                 Rudy Gobert  25:43  3   5   .600   0    0          2   4   
3                 Mike Conley  24:57  3   5   .600   2    3    .667  0   0   
4    Nickeil Alexander-Walker  23:30  2   6   .333   2    4    .500  1   2   
..                        ...    ... ..  ..    ...  ..  ...     ... ..  ..   
203                Zeke Nnaji   4:15  2   2  1.000   1    1   1.000  0   0   
204            Reggie Jackson   4:15  0   0          0    0          0   0   
205                 Ish Smith   4:15  0   1   .000   0    0          0   0   
206             Peyton Watson   4:15  0   1   .000   0    1    .000  0   0   
207             Vlatko Čančar   2:04  0   1   .000   0    1    .000  0   0   

     ... orb drb trb ast stl blk tov pf pts plus_minus  
0    .

In [11]:
df_selected.to_csv('output.csv', index=False)